In [ ]:
# Testing Cell
from aviary.utils.functions import get_model
from aviary.docs.tests.utils import glue_variable

csv_snippet = '```\n'
filename = 'aircraft_for_bench_FwFm.csv'
with open(get_model(filename)) as f_in:
    lines = f_in.readlines()
    l, s = [], 1
    for ii in range(7):
        l.append(lines[ii*2*s])
        s*=-1
    l.sort()
    csv_snippet+=''.join(l)

csv_snippet+='...\n```'
glue_variable('csv_snippet', csv_snippet)

# Vehicle Input .csv File and Phase_info Dictionary

## Vehicle Input .csv File

```{note}
This section is under development.
```

Aviary uses two main files to define the vehicle and the trajectory -- the vehicle .csv definition and a `phase_info` dictionary definition that defines the trajectory parameters.
You can think of the first file as describing the vehicle's physical properties and the second file as describing the trajectory's properties and optimization parameters.

The vehicle .csv file is structured as multiple rows, each containing a specific vehicle parameter's name, value, and units.
A portion of an example vehicle .csv file is shown below:

```{glue:md} csv_snippet
:format: myst
```

Depending on which analysis options you use with Aviary, you might need to define certain parameters within the vehicle .csv file.
The simplest way to get started is to use a provided example vehicle .csv file and modify it as needed for your specific vehicle.

In [ ]:
# Testing Cell
from aviary.utils.process_input_decks import create_vehicle
from aviary.utils.aviary_values import AviaryValues
from aviary.utils.process_input_decks import initialization_guessing
from aviary.api import Aircraft, LegacyCode
from aviary.interface.cmd_entry_points import _command_map
from aviary.docs.tests.utils import glue_variable, get_variable_name

default_guesses = '```\n'
vehicle_deck = AviaryValues()
_ , initialization_guesses = create_vehicle(vehicle_deck=vehicle_deck)
for key, val in initialization_guesses.items():
    default_guesses+=f"{key},{val}\n"
    glue_variable(key, md_code=True)
default_guesses+='```'
glue_variable('default_guesses', default_guesses)


glue_variable(get_variable_name(Aircraft.Design.RESERVE_FUEL_ADDITIONAL), md_code=True)
glue_variable(get_variable_name(Aircraft.Design.RESERVE_FUEL_FRACTION), md_code=True)

f2a = 'fortran_to_aviary'
_command_map[f2a];
glue_variable(f2a)
glue_variable(LegacyCode.GASP.name)

In vehicle .csv files that were created with the {glue:md}`fortran_to_aviary` converter that were created from {glue:md}`GASP` files, there is a section with the heading '# Initialization Guesses' that is used to initialize the trajectory. It contains the following keys along with default initialization values:

```{glue:md} default_guesses
:format: myst
```

The user can also specify these parameters with the prefix 'initialization_guesses:'
e.g. 'initialization_guesses:actual_takeoff_mass,150000' would set actual_takeoff_mass in the initialization_guesses dictionary to 150000.

When the initialization_guessing() method is called and wherever the initialization_guesses values are equal to 0, they are updated with calculated estimates based off the problem type (sizing, alternate, fallout) and mass, speed, range, thrust, and payload data specified in the vehicle input .csv file.

The initial guess of {glue:md}`reserves` is used to define the reserve fuel. Initially, its value can be anything larger than or equal to 0. There are two Aviary variables to control the reserve fuel in the model file (`.csv`):
- {glue:md}`Aircraft.Design.RESERVE_FUEL_ADDITIONAL`: the required fuel reserves: directly in lbm,
- {glue:md}`Aircraft.Design.RESERVE_FUEL_FRACTION`: the required fuel reserves: given as a proportion of mission fuel.

If the value of initial guess of {glue:md}`reserves` (also in the model file if any) is 0, the initial guess of reserve fuel comes from the above two Aviary variables.  Otherwise, it is determined by the parameter {glue:md}`reserves`:
- if `reserves > 10`, we assume it is the actual fuel reserves.
- if `0.0 <= reserves <= 10`, we assume it is the fraction of the mission fuel.

The initial guess of {glue:md}`reserves` is always converted to the actual design reserves (instead of reserve factor) and is used to update the initial guesses of {glue:md}`fuel_burn_per_passenger_mile` and {glue:md}`cruise_mass_final`.

In [ ]:
# Testing Cell
from aviary.interface.default_phase_info.height_energy import phase_info as HE_phase_info
from aviary.interface.default_phase_info.two_dof import phase_info as TwoDOF_phase_info
from aviary.interface.utils.check_phase_info import check_phase_info, HEIGHT_ENERGY, TWO_DEGREES_OF_FREEDOM
from aviary.docs.tests.utils import glue_keys
from aviary.interface.methods_for_level2 import AviaryProblem
from copy import deepcopy
import openmdao.api as om

check_phase_info(HE_phase_info, HEIGHT_ENERGY);
check_phase_info(TwoDOF_phase_info, TWO_DEGREES_OF_FREEDOM);

def get_completed_phase_info(filename, phase_info=None):
    prob = AviaryProblem()
    prob.load_inputs(filename, phase_info=phase_info)
    prob.check_and_preprocess_inputs()
    prob.add_phases()
    complete_phase_info = {}
    for phase in prob.phase_objects:
        phase_name, info = phase.to_phase_info()
        complete_phase_info[phase_name] = info
    return complete_phase_info


complex_cruise = deepcopy(HE_phase_info['cruise'])
#TypeError: EnergyPhase: complex_cruise: unsupported option: use_actual_takeoff_mass
# from aviary.mission.flops_based.ode.mission_ODE import MissionODE
# complex_cruise['user_options']['ode_class'] = MissionODE
# complex_cruise['user_options']['mass_f_cruise'] = 115000
# complex_cruise['user_options']['range_f_cruise'] = 3000
# complex_cruise['user_options']['solve_segments'] = True
# complex_cruise['user_options']['use_actual_takeoff_mass'] = True
# complex_cruise['user_options']['`fix_initial_time`'] = True

solved_alpha = deepcopy(HE_phase_info['cruise'])
solved_alpha['subsystem_options']['core_aerodynamics']['method'] = 'solved_alpha'
solved_alpha['subsystem_options']['core_aerodynamics']['aero_data'] = \
    "subsystems/aerodynamics/gasp_based/data/large_single_aisle_1_aero_free.txt"

pre_mission = deepcopy(HE_phase_info['pre_mission'])
pre_mission['linear_solver'] = om.DirectSolver()
pre_mission['nonlinear_solver'] = om.NewtonSolver()

custom_phase_info = {'pre_mission':pre_mission, 'complex_cruise':complex_cruise, 'solved_alpha':solved_alpha}

dummy_phase_info = {}
dummy_phase_info.update(HE_phase_info)
dummy_phase_info.update(get_completed_phase_info('aircraft_for_bench_FwFm.csv'))
dummy_phase_info.update(custom_phase_info)
dummy_phase_info.update(get_completed_phase_info('aircraft_for_bench_FwFm.csv', custom_phase_info))
dummy_phase_info.update(TwoDOF_phase_info)
dummy_phase_info.update(get_completed_phase_info('aircraft_for_bench_GwGm.csv'))
glue_keys(dummy_phase_info)


Loaded default phase_info for height_energy equations of motion


TypeError: EnergyPhase: complex_cruise: unsupported option: `fix_initial_time`

## Phase Info Dictionary

`phase_info` is a nested dictionary that Aviary uses for users to define their mission phases - how they are built, the design variables, constraints, connections, etc.

We will now discuss the meaning of the keys within the `phase_info` objects.

- If a key starts with `min_` or `max_` or ends with `_lower` or `_upper`, it is a lower or upper bound of a state variable. The following keys are not state variables:
  - {glue:md}`required_available_climb_rate`: the minimum rate of climb required from the aircraft at the top of climb (beginning of cruise) point in the mission. You don't want your available rate-of-climb to be 0 in case you need to gain altitude during cruise.
  - {glue:md}`EAS_limit`: the maximum descending EAS in knots.
  - {glue:md}`throttle`: the prescribed throttle setting. This is only used for `GASP` and `solved` missions.
- If a key ends with `_ref` or `_ref0` (except {glue:md}`duration_ref` and {glue:md}`initial_ref`), it is the unit-reference and zero-reference values of the control variable at the nodes. This option is invalid if opt=False. Note that it is a simple usage of  ref and ref0. We refer to [Dymos](https://openmdao.github.io/dymos/api/phase_api.html?highlight=ref0#add-state) for details.
- Some keys are for phase time only.
  - {glue:md}`duration_ref` is the unit-reference for phase time duration.
  - {glue:md}`duration_bounds` are the bounds (lower, upper) for the time duration of the phase.
  - {glue:md}`initial_ref` is the unit-reference for the initial value of time.
  - {glue:md}`initial_bounds`: the lower and upper bounds of initial time.
- If a key starts with `final_`, it is the final value of a state variable.
- If a key ends with `_constraint_eq`, it is an equality constraint.

- Keys related to altitude:
  - We use {glue:md}`final_altitude` to indicate the final altitude of the phase.
  - Meanwhile, {glue:md}`alt` is a key in acceleration phase parameter for altitude in `GASP` missions and {glue:md}`altitude` is a key in all other phases of all missions.

- Some keys are a boolean flag of True or False:
  - {glue:md}`input_initial`: the flag to indicate whether initial values of of a state (such as: altitude, velocity, mass, etc.) is taken.
  - {glue:md}`add_initial_mass_constraint`: the flag to indicate whether to add initial mass constraint
  - {glue:md}`clean`: the flag to indicate no flaps or gear are included.
  - {glue:md}`connect_initial_mass`: the flag to indicate whether the initial mass is the same as the final mass of previous phase.
  - {glue:md}`fix_initial`: the flag to indicate whether the initial state variables is fixed.
  <!-- - {glue:md}`fix_initial_time`: the flag to indicate whether the initial time is fixed. -->
  - {glue:md}`no_climb`: if True for the descent phase, the aircraft is not allowed to climb during the descent phase.
  - {glue:md}`no_descent`: if True for the climb phase, the aircraft is not allowed to descend during the climb phase.
  - {glue:md}`include_landing`: the flag to indicate whether there is a landing phase.
  - {glue:md}`include_takeoff`: the flag to indicate whether there is a takeoff phase.
  - {glue:md}`optimize_mass`: if True, the gross takeoff mass of the aircraft is a design variable.
  - {glue:md}`target_mach`: the flag to indicate whether to target mach number.
- {glue:md}`initial_guesses`: initial guesses of state variables.
- `COLLOCATION` related keys:
  - {glue:md}`num_segments`: the number of segments in transcription creation in Dymos. The minimum value is 1. This is needed if 'AnalysisScheme' is `COLLOCATION`.
  - {glue:md}`order`: the order of polynomials for interpolation in transcription creation in Dymos. The minimum value is 3. This is needed if 'AnalysisScheme' is `COLLOCATION`.
- Other Aviary keys:
  - {glue:md}`subsystem_options`: The {glue:md}`core_aerodynamics` key allows two methods: `computed` and `solved_alpha`. In case of `solved_alpha`, it requires an additional key {glue:md}`aero_data`.
  - {glue:md}`external_subsystems`: a list of external subsystems.
- other keys that are self-explanatory:
  - {glue:md}`clean`: a flag for low speed aero (which includes high-lift devices) or cruise aero (clean, because it does not include high-lift devices).
  - {glue:md}`EAS_target`: the target equivalent airspeed.
  - {glue:md}`initial_mach`: initial Mach number.
  - {glue:md}`linear_solver`:  provide an instance of a [LinearSolver](https://openmdao.org/newdocs/versions/latest/features/core_features/controlling_solver_behavior/set_solvers.html) to the phase. <!-- looks like this is only supported for pre_mission -->
  - {glue:md}`mach_cruise`: the cruise mach number.
  <!-- - {glue:md}`mass_f_cruise`: final cruise mass (kg). It is used as `ref` and `defect_ref` in cruise phase. -->
  - {glue:md}`nonlinear_solver`: provide an instance of a [NonlinearSolver](https://openmdao.org/newdocs/versions/latest/features/core_features/controlling_solver_behavior/set_solvers.html) to the phase. <!-- looks like this is only supported for pre_mission -->
  <!-- - {glue:md}`ode_class`: default to `MissionODE`. -->
  <!-- - {glue:md}`range_f_cruise`: final cruise range (m). It is used as `ref` and `defect_ref` in cruise phase. -->
  <!-- - {glue:md}`solve_segments`: False, 'forward', 'backward'. This is a Radau option. -->
  - {glue:md}`polynomial_control_order`: default to `None`.
  <!-- - {glue:md}`use_actual_takeoff_mass`: default to `False`. -->
  - {glue:md}`fix_duration`: default to `False`.
  - {glue:md}`solve_for_distance`: if True, use a nonlinear solver to converge the `distance` state variable to the desired value. Otherwise use the optimizer to converge the `distance` state.
  - {glue:md}`optimize_mach`: if True, the Mach number is a design variable.
  - {glue:md}`optimize_altitude`: if True, the altitude is a design variable.
  - {glue:md}`constraints`: a dictionary of user-defined constraints. The keys are the names of the constraints and the values are the keyword arguments expected by Dymos.

```{note}
Not all the keys apply to all phases. The users should select the right keys for each phase of interest. The required keys for each phase are defined in [check_phase_info](https://github.com/OpenMDAO/Aviary/blob/main/aviary/interface/utils.py) function. Currently, this function does the check only for `FLOPS` and `GASP` missions.
```

## Using custom phase builders

For the `height_energy`, you can use a user-defined phase builder.
The user-defined phase builder must inherit from `PhaseBuilderBase` and provide the `from_phase_info` and the `build_phase` methods.
The `from_phase_info` method is used to convert the `phase_info` dictionary into the inputs needed for the phase builder object.
The `build_phase` method is used to actually build and output the `Phase` object.

For examples of how to create a custom phase builder, see the `energy_phase.py` file.

```{note}
Using custom phase builders is a particularly advanced feature and is not recommended for most users.
```

In [ ]:
# Testing Cell
from aviary.mission.phase_builder_base import PhaseBuilderBase
PhaseBuilderBase.from_phase_info;
PhaseBuilderBase.build_phase;